# Evolução temporal de um ferromagneto utilizando o modelo Ising 2D
## Introdução teórica

Ferromagnetismo é a propriedade de certos materiais, como o ferro, de possuiram uma grande permeabilidade magnética, que por sua vez é o quanto a magnetização do material se alinha em resposta a um campo magnético externo. Alguns desses materiais podem se tornar imãs temporários. <br>
O modelo Ising $^{[1]}$ descreve um ferromagneto através de uma rede de particulas organizadas, normalmente, em uma grade, que interagem somente com os seus 4 vizinhos adjacentes. Este é um modelo de 2 estados, ou seja, cada particula pode assumir 1 de 2 valores discretos, nomeadamente spins $\pm 1$. <br>
A energia total do sistema é dada pelo seu Hamiltoniano que depende somente da configuração de spin de cada ponto da rede. <br>
\begin{equation} \tag{1}
H(\sigma) = - \sum_{i j}{J_{ij}\sigma_i \sigma_j} - \mu\sum_{j}{h_j\sigma_j} 
\end{equation} 
Onde: <br>
$J_{ij}$ é uma constante positiva para ferromagnetos, dependente de quais pontos da rede estão interagindo; <br>
$\sigma$ é o estado do sistema e $\sigma_i$ é o estado do i-nésimo ponto da rede; <br>
$\mu$ é o momento magnético; <br>
$h_i$ é o campo magnético externo que interage com o i-nésimo ponto da rede. <br>

Para cada estado $\sigma$ do sistema, podemos encontra-lo com uma probabilidade de configuração $P_{\beta}(\sigma)$, dada pela distribuição de Boltzmann, definida para alguma temperatura. <br>
\begin{equation} \tag{2}
P_{\beta}(\sigma) = \frac{e^{-\beta H(\sigma)}}{Z_{\beta}}
\end{equation}
Onde $\beta = \frac{1}{k_BT}$ e a constante de normalização é $Z_{\beta} = \sum_{\sigma}{e^{-\beta H(\sigma)}}$ <br>

### Modelo simplificado
De inicio, vamos analisar um caso simplificado onde $J_{ij} = J$ para todas as interações e o campo externo $h = 0$, o que nos leva a $$H_{red}(\sigma) = -J\sum_{i j }{\sigma_i \sigma_j}$$

Dessa forma, vamos iniciar a nossa simulação importando as bibliotecas padrão para exibição de gráficos, constantes e algumas funções já apresentadas. 



Usaremos as seguintes bibliotecas. 

In [1]:
import matplotlib.animation as animation
from matplotlib import pyplot as plt
from scipy import constants
import numpy as np
import random
%matplotlib qt


Abaixo definiremos o valor das constantes que usaremos:

In [2]:

J = 1
L = 500
T = 1
e = float(constants.e)
mu = 1
h = 1


E, 
então, definiremos as funções acima descritas, além da nossa rede, aqui chamada de grid.

In [3]:

grid = [[0]*L for i in range(L)]

def sigma(x,y):
    return grid[x%L][y%L]

def beta(T):
    return 1/(13*T)

def H_local(x,y):
    return -J * sigma(x,y) * (sigma(x-1,y-1) + sigma(x-1,y+1) + sigma(x+1,y-1) + sigma(x+1,y+1))


Onde 
$H_{local}(\sigma_{xy})$ é a energia da interação do ponto localizado nas coordenadas x e y com os seus vizinhos adjacentes. <br>
Estaremos trabalhando com uma condição de contorno periódica, introduzida sempre que acessarmos o estado de um dos pontos da rede através da função sigma(x,y). 
Essa condição de contorno é necessária para que todos os pontos estejam interagindo com 4 vizinhos, emulando um ambiente infinito, apesar de que na prática estamos em uma geometria fechada. <br>
Também inicializaremos o nosso grid e vamos randomiza-lo, abaixo, com valores de -1 ou 1.

In [4]:
def RandomizeGrid():
    for i in range(0,L):
        for j in range(0,L):
            grid[i][j] = random.randrange(-1,2,2)
            
RandomizeGrid()
plt.imshow(grid, cmap='gray')
plt.show()


$ $

## Algorítimo de Metropolis
No nosso sistema temos 500 pontos, cada um com 2 estados distintos, totalizando um total de $2^{500}$ estados possíveis. 
Pela enorme quantidade de estados, torna-se impraticável calcular $Z_{\beta}$. <br> 
Dessa forma, precisamos de um método onde não é necessário a normalização da função de probabilidade e somos incentivados a utilizar um dos métodos de Monte Carlo $^{[3]}$ para essa simulação. Mais especificamente, estaremos utilizando o algorítimo de Metropolis $^{[3]}$. <br>
Este algorítimo consiste em escolher uma função de probabilidade de seleção $g(u, v)$ que indica a probabilidade de um estado v ser selecionado dado que estamos no estado u. É interessante que essa função seja simétrica, ou seja $g(u, v) = g(v, u)$, já que a quebra de simetria será introduzida em uma função posterior, podendo a função g ser uma distribuição gaussina, por exemplo, tornando estados mais próximos de u mais prováveis de serem escolhidos. <br>
Como estaremos realizando a iteração na simulação escolhendo um ponto por vez, qualquer estado seguinte v será igualmente e imediatamente próximo ao atual u, de forma que devemos escolher $g(u, v) = 1/L^2$, por ser mais coerente. <br>
Após um dos pontos da rede ser escolhido para que ocorra um flip no valor do spin, manteremos a maximização da entropia, respeitando a equação (2), atravez da introdução de uma função de probabilidade de aceitação $A(u,v)$ que define a probabilidade do novo estado v ser aceito dado que estamos no estado u. Não é surpresa que tal função seja exatamente a própria equação (2) quando desconhecemos o estado atual u. <br>
O modelo Ising pode ser visto como uma cadeia de Markov, um sistema estocástico onde cada evento de sequencia de eventos depende apenas do evento anterior. Quando realizamos uma transição do estado u para o v, através do flip (opcional) de um único spin, a probabilidade desse novo estado ser um estado especifico depende de qual era o estado anterior (a entropia tende a crescer).
Podemos usar o teorema de Bayes para encontrar esse valor:
$$P(v|u)=\frac{P(u|v)P(v)}{P(u)}$$
Onde $P(v|u) = P(u,v)$ e $P(u) = P_{\beta}(u)$ <br>
Mas, como podemos decompor $P(u,v) = g(u, v)A(u, v)$, isso nos leva a:
$$\frac{A(u,v)}{A(v,u)} = \frac{P_{\beta}(v)}{P_{\beta}(u)} = \frac{\frac{1}{Z}e^{-\beta H_v}}{\frac{1}{Z}e^{-\beta H_u}} = e^{-\beta (H_v-H_u)}$$
A função que obedece a essa relação pode ser descrita como a seguir:
$$A(u,v) = \begin{cases} e^{-\beta (H_v-H_u)}, &\text{se } H_v - H_u \text{ > 0} \\ 1, &\text{caso contrário} \end{cases} $$
Note que a função de probabilidade de aceitação dependerá apenas da diferença de energia entre os estados, que por sua vez depende apenas da interação do ponto da rede invertido com os seus vizinhos adjacentes. <br>
Vamos chamar $H_v-H_u = dH$, que pode ser obtido por:
$$dH_i = 2J\sigma_i(\sum_{j}{\sigma_j}) \tag{3}$$
Onde j itera nas vizinhanças de i.

Abaixo
definiremos as funções que executam as equações acima citadas

In [5]:
def dHNoField(position):
    (x, y) = position
    return 2 * J * sigma(x,y) * (sigma(x-1,y) + sigma(x+1,y) + sigma(x,y-1) + sigma(x,y+1))

def Selection():
    x = random.randint(0,L-1)
    y = random.randint(0,L-1)
    return (x, y)

def A(dH):
    if dH > 0:
          return np.exp(-dH/T)
    else:
        return 1
    
def Acceptance(position, dHFunc):
    random_num = random.random()
    if random_num <= A(dHFunc(position)):
        return True
    else:
        return False

def flip(position):
    (x, y) = position
    grid[x%L][y%L] *= -1
    
def Iterations(iterations_per_frame, dHFunc):
    for i in range(L*iterations_per_frame):
        position = Selection()
        result = Acceptance(position, dHFunc)
        if result:
            flip(position)



A 
função Iterations é a responsável pela evolução do nosso sitema. Defini, arbitrariamente, um passo base de tamanho L, ou seja, 500 comparações em cada passo.
Abaixo montamos a animação correspondente ao estado do sistema ao longo de number_of_frames*iterations_per_frame passos.

In [6]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.axis('off')
ax.set_title(label = "Sem campo", fontsize=20)

RandomizeGrid()

number_of_frames = 3000
iterations_per_frame = 30 #tamanho da iteração = L

def GetFrames(number_of_frames, dHFunc):
    im = plt.imshow(grid, cmap= 'gray')
    ims = []
    ims.append([im])
    for i in range(number_of_frames-1):
        Iterations(iterations_per_frame, dHFunc)
        im = plt.imshow(grid, cmap= 'gray')
        ims.append([im])
    return ims

ims_no_field = GetFrames(number_of_frames, dHNoField)

animation_no_field = animation.ArtistAnimation(fig, ims_no_field, interval=20)

animation_no_field.save("No_field.gif")

![No_field](No_field.gif)

### Sistema sob ação de um campo
Na nossa descrição acima trabalhamos com um hamiltoniano sem a ação de um campo magnético externo. Agora, vamos descrever como o sistema se comporta sob a ação de um campo. <br>
A partir de (1) e, novamente, considerando que $J_{ij} = J$ para todo ij e considerando um campo homogêneo $h_j = h$, obtemos o hamiltoniano:
$$H(\sigma) = - J\sum_{i j}{\sigma_i \sigma_j} - \mu h\sum_{j}{\sigma_j} $$
Observe que, sob a ação de um campo externo, o hamiltoniano não é mais invariável a uma inversão de todos os spins da rede. <br>
Assim como fizemos com (3), a nova expressão para a variação de energia será:
$$dH_i = 2J\sigma_i(\sum_{j}{\sigma_j}) + 2\mu h\sigma_i $$
Onde o termo referente ao campo depende apenas do ponto da rede que foi invertido. Podendo ser reduzido para:
$$dH_i = 2\sigma_i(J(\sum_{j}{\sigma_j}) + \mu h \tag{4}) $$
Ou seja, o campo irá introduzir um *bias* na decisão se um spin deve ser invertido ou não. <br>
Podemos prosseguir com a simulação igual a como foi feito anteriormente, porém usando agora o novo hamiltoniano.



In [7]:
def dHField(position):
    (x, y) = position
    return dHNoField(position) + 2*mu*h*sigma(x,y)

RandomizeGrid()

ax.set_title(label = "Com campo", fontsize=20)

ims_with_field = GetFrames(number_of_frames = 400, dHFunc = dHField)

animation_with_field = animation.ArtistAnimation(fig, ims_with_field, interval=20)

animation_with_field.save("With_field.gif")


![With_field](With_field.gif)

## Referências 
[1] Gallavotti, G. (1999), Statistical mechanics, Texts and Monographs in Physics <br>
[2] https://en.wikipedia.org/wiki/Ising_model <br>
[3] Newman, M.E.J.; Barkema, G.T. (1999). Monte Carlo Methods in Statistical Physics. Clarendon Press. ISBN 9780198517979. <br>